In [54]:
# !pip install torch==2.1.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.1.0+cu121
# !pip install --upgrade sympy
# !pip uninstall -y transformers
# !pip install transformers
# !pip install bitsandbytes

In [2]:
from google.colab import drive
import numpy as np
from huggingface_hub import notebook_login
import torch
import time
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import warnings
warnings.filterwarnings('ignore')

In [3]:
notebook_login()

In [87]:
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/PW_NLP

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/PW_NLP'
/content/gdrive/MyDrive/PW_NLP


In [5]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

!nvidia-smi

12.1
True
Mon Jan 27 21:24:11 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+---------------

In [7]:
model_card = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.pad_token = tokenizer.eos_token

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

generation_config = model.generation_config
generation_config.max_new_tokens = 500
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.temperature = 0.0
generation_config.num_return_sequences = 1

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [88]:
def call_model(model, input_ids_prompt, generation_config):
    start_time = time.time()
    with torch.no_grad():
      output = model.generate(
          **input_ids_prompt,
          max_new_tokens=generation_config.max_new_tokens,
          eos_token_id=generation_config.eos_token_id,
          pad_token_id=generation_config.pad_token_id,
          temperature=generation_config.temperature,
          num_return_sequences=generation_config.num_return_sequences
      )
      end_time = time.time()

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text, round(end_time-start_time, 2)

In [89]:
import re

def parse_response(text):
    # Define regex patterns for extracting the title, description, and keywords
    title_pattern = r"'title':\s*'(.*?)'"
    description_pattern = r"'description':\s*'(.*?)'"
    keywords_pattern = r"'keywords':\s*\[(.*?)\]"

    # Extract title, description, and keywords using regex
    title_match = re.search(title_pattern, text)
    description_match = re.search(description_pattern, text)
    keywords_match = re.search(keywords_pattern, text)

    # Check if all parts were found
    if title_match and description_match and keywords_match:
        title = title_match.group(1)
        description = description_match.group(1)
        # Clean up the keywords list and join them into a string
        keywords = ', '.join([kw.strip("'") for kw in keywords_match.group(1).split(',')])

        # Return the formatted string
        return f"{title}. {description} {keywords}"
    else:
        return "Error: Could not extract information"

In [ ]:
import os

template = """
You are a finance expert who specializes in analyzing tables.

### Task ###
Summarize the table in a dictionary with the following keys:
- 'title': A concise title for the table.
- 'description': A brief description of the table's contents and key trends.
- 'keywords': A list of relevant keywords.

**Note**: Do not return the table, prompt, or any extra text. Return only the dictionary.

### Table ###
"""

# Check if the results file are already there, and continue in case
if os.path.exists('Summaries.npy'):
    summaries_npy = np.load('Summaries.npy', allow_pickle='TRUE').item()
    print("Summaries.npy already created!\t It contains: ", len(summaries_npy), " summary tables ", list(summaries_npy.keys()))
else:
    summaries_npy = {}

if os.path.exists('Timing.npy'):
    timing_npy = np.load('Timing.npy', allow_pickle='TRUE').item()
    print("timing.npy already created!\t It contains: ", len(timing_npy), " summary tables ")
else:
    timing_npy = {}

count = 0
print("\n\n")

# Load the extracted tables to compute the summaries
table_dict = np.load('extracted_tables.npy', allow_pickle='TRUE').item()

for key, value in table_dict.items():
    print(f"\tKey: {key}")
    # Check if the table has already been processed
    if key in summaries_npy:
        print(f"\t\tSkipping {key}, already processed ...\n")
        continue

    table = value[0]
    prompt = template + table
    # Tokenize the prompt
    input_ids_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    # Call the model
    model_response, total_time = call_model(model, input_ids_prompt, generation_config)
    # Parse the response
    response_text = model_response[len(prompt):]
    parsed_resp = parse_response(response_text)
    print(parsed_resp)
    print("\n\n")

    summaries_npy[key] = parsed_resp
    np.save('Summaries.npy', summaries_npy)

    timing_npy[key] = total_time
    np.save('Timing.npy', timing_npy)

    # count += 1
    # if count == 2:
    #     break

Summaries.npy already created!	 It contains:  6  summary tables  ['dd49713c6', 'dd497692a', 'dd498d03a', 'dd498655a', 'dd4988030', 'dd496f026']
timing.npy already created!	 It contains:  2  summary tables 



	Key: dd49713c6
		Skipping dd49713c6, already processed ...
	Key: dd497692a
		Skipping dd497692a, already processed ...
	Key: dd498d03a
		Skipping dd498d03a, already processed ...
	Key: dd498655a
		Skipping dd498655a, already processed ...
	Key: dd4988030
		Skipping dd4988030, already processed ...
	Key: dd496f026
		Skipping dd496f026, already processed ...
	Key: dd497ad9a
Financial Performance Over Three Years. This table shows the financial performance of a company over three years. Key trends include a steady increase in sales, a significant increase in operating profit, and a substantial increase in net profit. sales,  'operating profit,  'net profit,  'financial performance,  'trends,  'increase



	Key: dd4970aa2
Stock Purchase Summary. This table shows the total number of s

In [101]:
summaries_npy = np.load('Summaries.npy', allow_pickle='TRUE').item()
summaries_npy

{'dd49713c6': "Financial Risk Analysis. This table shows the level 3 assets (liabilities) and range of significant unobservable inputs for various financial risk factors as of December 2016 and 2017. finance,  'risk,  'analysis,  'assets,  'liabilities,  'unobservable inputs,  '2016,  '2017,  'interest rates,  'correlation,  'volatility,  'credit,  'credit spreads,  'upfront credit points,  'recovery rates,  'currencies,  'commodities,  'natural gas spread,  'oil spread,  'equities",
 'dd497692a': "Restricted Stock Units (RSUs) Over Time. This table shows the number of RSUs granted, vested, forfeited, and the weighted average grant date fair value for each year from 2008 to 2010. RSUs,  'granted,  'vested,  'forfeited,  'fair value,  '2008,  '2009,  '2010",
 'dd498d03a': "Yearly Amount. This table shows the yearly amount from 2014 to 2018, with a significant decrease in the amount after 2018. The total amount from 2014 to 2018 is $1286. year,  'amount,  'decrease,  'total,  '2014,  '20

In [103]:
timing_npy = np.load('Timing.npy', allow_pickle='TRUE').item()
timing_npy

{'dd4988030': 26.37, 'dd496f026': 40.17}